# Content and Objective

+ Show how frequency response is obtained based on poles and zeros
+ User may define poles and zeros 
+ Amplitude response and phase response will be plotted

In [31]:
#importing
import numpy as np
from scipy import signal as sig

import matplotlib.pyplot as plt
import ipywidgets as widgets


# Reminder



The magnitude of an LTI system is given by:


$$\left|H(\Omega)\right|=\left|H_0\right|\frac{\prod_{\mu=1}^p |e^{j\Omega}-z_{0,\mu}|}{\prod_{\nu=1}^q |e^{j\Omega}-z_{\infty,\nu}|}$$
 

+ Numerator is determined as the product of distances between the current frequency point $\mathrm{e}^{\mathrm{j}\Omega}$ and the zeros
+ Denominator is determined as the product of distances between the current frequency point $\mathrm{e}^{\mathrm{j}\Omega}$ and the poles

Accordingly, the phase is obtained by the formula:

$$\varphi(\Omega)=\arg \left( H_0\frac{\prod_{\mu=1}^p \left( e^{j\Omega}-z_{0,\mu}\right)}{\prod_{\nu=1}^q \left( e^{j\Omega}-z_{\infty,\nu}\right)}\right)$$

# Simulation

In [32]:
# function for getting the magnitude
def magnitude( omega, zeros, poles):
    '''
    IN: (normalized) frequency at which the magnitude is to be determined
        zeros
        poles
    
    OUT: magnitude as numpy array
    '''
    
    # get points on the unit circle where magnitude should be determined
    points_on_circle = np.exp( 1j * omega)
    
    # motivate why you are doing this!
    points_on_circle_matrix_nominator = np.tile(points_on_circle, (len(zeros), 1))
    points_on_circle_matrix_denominator = np.tile( points_on_circle, (len(poles), 1))

    # distances to zeros
    differences_nominator = points_on_circle_matrix_nominator - zeros[:,np.newaxis]
    
    # distances to poles
    differences_denominator = points_on_circle_matrix_denominator - poles[:,np.newaxis]

    # axis=0 to build product along columns! (each column is one frequency)
    nominator = np.prod( differences_nominator, axis=0)
    denominator = np.prod( differences_denominator, axis=0)

    # return result
    return np.abs( nominator / denominator )


In [33]:
# function for getting the phase
def phase(omega, zeros, poles):
    '''
    IN: frequency at which the magnitude is to be determined
        zeros
        poles
    
    OUT: phase as numpy array
    '''
     
    # get points on the unit circle where magnitude should be determined
    points_on_circle = np.exp( 1j * omega )

    # motivate why you are doing this!
    points_on_circle_matrix_nominator = np.tile( points_on_circle, (len(zeros), 1))
    points_on_circle_matrix_denominator = np.tile( points_on_circle, (len(poles), 1))

    # distances to zeros
    differences_nominator = points_on_circle_matrix_nominator - zeros[:, np.newaxis]
    
    # distances to zeros
    differences_denominator = points_on_circle_matrix_denominator - poles[:, np.newaxis]

    # axis=0 to build product along columns! (each column is one frequency)
    nominator = np.prod( differences_nominator, axis=0)
    denominator = np.prod( differences_denominator, axis=0)
    
    # return result
    return np.angle( nominator / denominator )


### Define Zeros and Poles

In [34]:
poles = np.array( [ 0.25 + 0.8j, 0.25 - 0.8j, 0.5])
# poles = np.array([ -.9 + .1j, -.9 - .1j, .95 ] )


zeros = np.array([0, 0, 0])

### Get Frequencies and Full Amplitude Response

In [35]:
# define quantization in omega and get sampled omega values
delta_omega = 0.01
omega_full = np.arange( -np.pi, np.pi, delta_omega )

# get magnitude and phase
amplitudengang_full = magnitude( omega_full, zeros=zeros, poles=poles )
#phasengang_full = phase( omega_full, zeros=zeros, poles=poles )


# Plotting

+ Zeros-poles, amplitude response, and phase response are shown
+ User may select the relevant frequency $\Omega_0$ by a slider
+ **NOTE:** frequency is in $[-3.1415, 3.1415]$
+ Afterwards, amplitude and phase response are given in $[-\pi, \Omega_0]$


In [36]:
# function for interactive plotting of magnitude and phase
def plot_amplitudengang_phasengang( Omega ):
    '''
    IN: upper frequency up to which magnitude and phase are drawn
    
    OUT: -
    '''
    # define quantization and overall omega for full circle
    delta_omega = 0.01
    omega_full = np.arange(-np.pi, np.pi, delta_omega)
    
    # define omega for frequencies -pi ... Omega (used for intermediate plotting)
    omega = np.arange( -np.pi, Omega, delta_omega)
    
    # amplitude and frequency resposnse for full circle
    amplituden_gang = magnitude( omega=omega, zeros=zeros, poles=poles)
    phasen_gang = phase( omega=omega, zeros=zeros, poles=poles)

    # init figure and plot unit circle
    fig, ax = plt.subplots(1, 3, figsize=(30, 10))
    point_on_circle = np.exp(1j*Omega)


    # sub-figure 0 shows 
    #    unit circle
    #    current point
    #    zeros and poles, 
    #    distances of current point to zeros and poles
    ax[0].plot( np.exp( 1j * omega_full ).real, np.exp( 1j * omega_full ).imag)  
    ax[0].scatter( point_on_circle.real, point_on_circle.imag, c='r')
      
    ax[0].scatter(zeros.real, zeros.imag, marker='o', c='b', s=180)
    ax[0].scatter(poles.real, poles.imag, marker='x', c='y', s=180)
    
    for pole in poles:
        ax[0].plot([point_on_circle.real, pole.real], [
                   point_on_circle.imag, pole.imag], '-', c='y')
    for zero in zeros:
        ax[0].plot([point_on_circle.real, zero.real], [
            point_on_circle.imag, zero.imag], '-', c='b')

    # sub-figure 1 shows amplitude response
    # and does a lot of parametrization to obtain a nice figure
    ax[1].plot(omega, amplituden_gang)    

    ax[1].set_xlim(-np.pi, np.pi)
    ax[1].set_ylim(0, amplitudengang_full.max()*1.1)
    ax[1].set_title('Amplitudengang')
    ax[1].set_xlabel('$\Omega$')
    ax[1].set_ylabel('$|A(\Omega$)')
    
    
    # sub-figure 2 shows phase response
    # and does a lot of parametrization to obtain a nice figure
    ax[2].plot(omega, phasen_gang)
    
    ax[2].set_xlim(-np.pi, np.pi)
    ax[2].set_ylim(-np.pi/2*1.1, np.pi/2*1.1)
    ax[2].set_title('Phasengang')
    ax[2].set_xlabel('$\Omega$')
    ax[2].set_ylabel('phase$(A(\Omega$))')
    
    # plt.scatter([1], [1]) so kann man einen weiteren Punkt ergänzen!
    plt.show()


In [37]:
# define widget and apply 
w = widgets.interact(plot_amplitudengang_phasengang, Omega=widgets.FloatSlider(
    min=-np.pi, max=np.pi, step=0.01, value=-np.pi, continuous_update=False))

interactive(children=(FloatSlider(value=-3.141592653589793, continuous_update=False, description='Omega', max=…

# Plotting

+ Zeros-poles, amplitude response, and phase response are shown
+ Frequency $\Omega_0$ dynamically takes all values in $[-\pi, \pi]$
+ Amplitude and phase response are provided as animation

In [38]:
# function for dynamic animation 
def plot_amplitudengang_phasengang_animation( degree ):
    '''
    IN: 
    
    OUT: -
    '''
    
    # map degree to rad
    Omega = ( degree - 180 ) / 360 * 2 * np.pi
    
    # define quantization and overall omega for full circle
    delta_omega = 0.01
    omega_full = np.arange(-np.pi, np.pi, delta_omega)
    
    # define omega for frequencies -pi ... Omega (used for intermediate plotting)
    omega = np.arange( -np.pi, Omega, delta_omega)

    # Use the predefined method to calculate both amplitude and phase:
    amplituden_gang = magnitude(omega=omega, zeros=zeros, poles=poles)
    phasen_gang = phase(omega=omega, zeros=zeros, poles=poles)

    # init figure and define point on circle for current Omega
    fig, ax = plt.subplots(1, 3, figsize=(30, 10))

    point_on_circle = np.exp(1j * Omega)

    
    # sub-figure 0 shows 
    #    unit circle
    #    current point
    #    zeros and poles, 
    #    distances of current point to zeros and poles
    ax[0].plot( np.exp(1j*omega_full).real, np.exp(1j*omega_full).imag)  
    ax[0].scatter( point_on_circle.real, point_on_circle.imag, c='r')
    
    ax[0].scatter(zeros.real, zeros.imag, marker='o', c='b', s=180)
    ax[0].scatter(poles.real, poles.imag, marker='x', c='y', s=180)
    
    for pole in poles:
        ax[0].plot([point_on_circle.real, pole.real], [
                   point_on_circle.imag, pole.imag], '-', c='y')
    for zero in zeros:
        ax[0].plot([point_on_circle.real, zero.real], [
            point_on_circle.imag, zero.imag], '-', c='b')
        
    # sub-figure 1 shows amplitude response
    # and does a lot of parametrization to obtain a nice figure   
    ax[1].plot(omega, amplituden_gang)

    ax[1].set_xlim(-np.pi, np.pi)
    ax[1].set_ylim(0, amplitudengang_full.max()*1.1)
    ax[1].set_title('Amplitude response')
    ax[1].set_xlabel('$\Omega$')
    ax[1].set_ylabel('$|A(\Omega$)')
    
    # sub-figure 2 shows phase response
    # and does a lot of parametrization to obtain a nice figure   
    ax[2].plot(omega, phasen_gang)
    
    ax[2].set_xlim(-np.pi, np.pi)
    ax[2].set_ylim(-np.pi/2*1.1, np.pi/2*1.1)
    ax[2].set_title('Phase response')
    ax[2].set_xlabel('$\Omega$')
    ax[2].set_ylabel('phase$(A(\Omega$))')

In [39]:
widgets.interact( plot_amplitudengang_phasengang_animation, degree=widgets.Play(
    min=0, max=360) )#, wait=True, animation_duration=600)



interactive(children=(Play(value=0, description='degree', max=360), Output()), _dom_classes=('widget-interact'…

<function __main__.plot_amplitudengang_phasengang_animation(degree)>